# Deliverable 1. Statistics - Multivariate Data Preprocessing of a Time Series

### Mariana Castro Payns - A01706038

#### *Initial configuration*

In [1]:
# Import necessary libraries 
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from statsmodels.stats.outliers_influence import variance_inflation_factor
import statsmodels.formula.api as sm

In [2]:
# Import dataset
df = pd.read_csv('us2022q2a.csv')
df

,firm,q,revenue,cogs,sgae,otheropexp,extraincome,finexp,incometax,totalassets,totalliabilities,shortdebt,longdebt,stockholderequity,adjprice,originalprice,sharesoutstanding,fiscalmonth,year,cto
0,A,2000q1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,63.761670,104.0000,452000.000,NaN,2000.0,1.0
1,A,2000q2,2485000.0,1261000.0,1.010000e+06,0.0,42000.000000,0.000,90000.0,7321000.000,2679000.000,512000.000,0.000,4642000.000,45.215607,73.7500,452271.967,6.0,2000.0,2.0
2,A,2000q3,2670000.0,1369000.0,1.091000e+06,0.0,28000.000000,0.000,83000.0,7827000.000,2925000.000,528000.000,0.000,4902000.000,30.003238,48.9375,453014.579,9.0,2000.0,3.0
3,A,2000q4,3372000.0,1732000.0,1.182000e+06,0.0,10000.000000,0.000,163000.0,8425000.000,3160000.000,830000.000,0.000,5265000.000,33.566841,54.7500,456366.381,12.0,2000.0,4.0
4,A,2001q1,2841000.0,1449000.0,1.113000e+06,0.0,-6000.000000,0.000,119000.0,9208000.000,3667000.000,556000.000,0.000,5541000.000,18.840347,30.7300,456769.737,3.0,2001.0,1.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
324442,ZYNE,2021q3,0.0,0.0,1.021065e+04,0.0,-376.636750,-5.038,0.0,89996.170,11309.171,207.635,411.237,78686.999,4.240000,4.2400,41251.537,9.0,2021.0,3.0
324443,ZYNE,2021q4,0.0,0.0,8.836436e+03,0.0,16.937906,-4.433,0.0,81171.507,10258.173,209.068,353.694,70913.334,2.880000,2.8800,41217.537,12.0,2021.0,4.0
324444,ZYNE,2022q1,0.0,0.0,8.903915e+03,0.0,317.252110,-96.044,0.0,74381.029,9214.059,210.512,295.754,65166.970,2.050000,2.0500,42447.037,3.0,2022.0,1.0
324445,ZYNE,2022q2,0.0,0.0,9.168770e+03,0.0,-775.927860,-91.691,0.0,67006.959,9765.086,211.965,237.414,57241.873,1.140000,1.1400,43595.959,6.0,2022.0,2.0


In [3]:
# Define name of columns and delete the ones that we wont be using
columns = ["firm", "Name", "N", "Class", "Country of Origin", "Type of Asset", "Sector NAICS level 1", "Exchange / Src", "Sector Economatica", "Sector NAICS last available", "partind"]
firms = pd.read_csv("usfirms2022.csv", names=columns)
firms = firms.drop(["N", "Name","Class", "Country of Origin", "Type of Asset", "Exchange / Src", "Sector Economatica", "Sector NAICS last available", "partind"], axis = 1)

In [4]:
# Merge data sets to get all the information we need
df = pd.merge(df,firms, on = 'firm')
df

,firm,q,revenue,cogs,sgae,otheropexp,extraincome,finexp,incometax,totalassets,...,shortdebt,longdebt,stockholderequity,adjprice,originalprice,sharesoutstanding,fiscalmonth,year,cto,Sector NAICS level 1
0,A,2000q1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,63.761670,104.0000,452000.000,NaN,2000.0,1.0,Manufacturing
1,A,2000q2,2485000.0,1261000.0,1.010000e+06,0.0,42000.000000,0.000,90000.0,7321000.000,...,512000.000,0.000,4642000.000,45.215607,73.7500,452271.967,6.0,2000.0,2.0,Manufacturing
2,A,2000q3,2670000.0,1369000.0,1.091000e+06,0.0,28000.000000,0.000,83000.0,7827000.000,...,528000.000,0.000,4902000.000,30.003238,48.9375,453014.579,9.0,2000.0,3.0,Manufacturing
3,A,2000q4,3372000.0,1732000.0,1.182000e+06,0.0,10000.000000,0.000,163000.0,8425000.000,...,830000.000,0.000,5265000.000,33.566841,54.7500,456366.381,12.0,2000.0,4.0,Manufacturing
4,A,2001q1,2841000.0,1449000.0,1.113000e+06,0.0,-6000.000000,0.000,119000.0,9208000.000,...,556000.000,0.000,5541000.000,18.840347,30.7300,456769.737,3.0,2001.0,1.0,Manufacturing
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
323811,ZYNE,2021q2,0.0,0.0,9.838494e+03,0.0,-117.528220,-5.943,0.0,98195.904,...,206.211,468.385,87699.257,5.290000,5.2900,41251.537,6.0,2021.0,2.0,Manufacturing
323812,ZYNE,2021q3,0.0,0.0,1.021065e+04,0.0,-376.636750,-5.038,0.0,89996.170,...,207.635,411.237,78686.999,4.240000,4.2400,41251.537,9.0,2021.0,3.0,Manufacturing
323813,ZYNE,2021q4,0.0,0.0,8.836436e+03,0.0,16.937906,-4.433,0.0,81171.507,...,209.068,353.694,70913.334,2.880000,2.8800,41217.537,12.0,2021.0,4.0,Manufacturing
323814,ZYNE,2022q1,0.0,0.0,8.903915e+03,0.0,317.252110,-96.044,0.0,74381.029,...,210.512,295.754,65166.970,2.050000,2.0500,42447.037,3.0,2022.0,1.0,Manufacturing


In [5]:
df = df.groupby(["firm"]).take([-2])
df = df.reset_index()
df

,firm,level_1,q,revenue,cogs,sgae,otheropexp,extraincome,finexp,incometax,...,shortdebt,longdebt,stockholderequity,adjprice,originalprice,sharesoutstanding,fiscalmonth,year,cto,Sector NAICS level 1
0,A,88,2022q1,1674000.0,764000.0,5.340000e+05,0.0,-37000.00000,20000.000,36000.0,...,0.000,2.730000e+06,5154000.00,131.888962,132.3300,300113.377,3.0,2022.0,1.0,Manufacturing
1,AA,178,2022q1,3293000.0,2181000.0,2.130000e+05,125000.0,-70000.00000,25000.000,210000.0,...,1000.000,1.727000e+06,6257000.00,89.694160,90.0300,185403.032,3.0,2022.0,1.0,Manufacturing
2,AAIC,268,2022q1,8470.0,4773.0,0.000000e+00,0.0,-4111.00000,0.000,2287.0,...,NaN,1.639200e+05,218097.00,3.470000,3.4700,35016.392,3.0,2022.0,1.0,Finance and Insurance
3,AAL,358,2022q1,8899000.0,0.0,1.062200e+07,0.0,92000.00000,455000.000,-451000.0,...,2382000.000,3.546100e+07,-8940000.00,18.250000,18.2500,649160.117,3.0,2022.0,1.0,Transportation and Warehousing
4,AAME,448,2022q1,51608.0,0.0,4.781200e+04,0.0,0.00000,0.000,954.0,...,NaN,3.373800e+04,126423.00,3.112825,3.1300,20378.576,3.0,2022.0,1.0,Finance and Insurance
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3595,ZVIA,323454,2022q1,38034.0,23413.0,2.327500e+04,8901.0,6669.00000,0.000,12.0,...,608.000,4.840000e+02,93861.00,4.570000,4.5700,66686.135,3.0,2022.0,1.0,Manufacturing
3596,ZVO,323544,2022q1,61633.0,39829.0,2.903600e+04,0.0,-127.00000,0.000,78.0,...,0.000,0.000000e+00,11267.00,0.820100,0.8201,34054.879,3.0,2022.0,1.0,Educational Services
3597,ZWS,323634,2022q1,239600.0,137700.0,5.690000e+04,1100.0,1100.00000,4800.000,10000.0,...,5600.000,5.403000e+05,193700.00,35.281528,35.4000,125782.456,3.0,2022.0,1.0,Manufacturing
3598,ZY,323724,2022q1,4791.0,12455.0,5.608200e+04,-130.0,-532.00000,7994.000,-26.0,...,57845.000,1.811680e+05,344064.00,2.890000,2.8900,103120.808,3.0,2022.0,1.0,"Professional, Scientific, and Technical Services"


In [6]:
df = df.drop(columns = ["level_1"])

In [7]:
df

,firm,q,revenue,cogs,sgae,otheropexp,extraincome,finexp,incometax,totalassets,...,shortdebt,longdebt,stockholderequity,adjprice,originalprice,sharesoutstanding,fiscalmonth,year,cto,Sector NAICS level 1
0,A,2022q1,1674000.0,764000.0,5.340000e+05,0.0,-37000.00000,20000.000,36000.0,1.032700e+07,...,0.000,2.730000e+06,5154000.00,131.888962,132.3300,300113.377,3.0,2022.0,1.0,Manufacturing
1,AA,2022q1,3293000.0,2181000.0,2.130000e+05,125000.0,-70000.00000,25000.000,210000.0,1.598800e+07,...,1000.000,1.727000e+06,6257000.00,89.694160,90.0300,185403.032,3.0,2022.0,1.0,Manufacturing
2,AAIC,2022q1,8470.0,4773.0,0.000000e+00,0.0,-4111.00000,0.000,2287.0,9.208830e+05,...,NaN,1.639200e+05,218097.00,3.470000,3.4700,35016.392,3.0,2022.0,1.0,Finance and Insurance
3,AAL,2022q1,8899000.0,0.0,1.062200e+07,0.0,92000.00000,455000.000,-451000.0,6.740100e+07,...,2382000.000,3.546100e+07,-8940000.00,18.250000,18.2500,649160.117,3.0,2022.0,1.0,Transportation and Warehousing
4,AAME,2022q1,51608.0,0.0,4.781200e+04,0.0,0.00000,0.000,954.0,3.750310e+05,...,NaN,3.373800e+04,126423.00,3.112825,3.1300,20378.576,3.0,2022.0,1.0,Finance and Insurance
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3595,ZVIA,2022q1,38034.0,23413.0,2.327500e+04,8901.0,6669.00000,0.000,12.0,1.164800e+05,...,608.000,4.840000e+02,93861.00,4.570000,4.5700,66686.135,3.0,2022.0,1.0,Manufacturing
3596,ZVO,2022q1,61633.0,39829.0,2.903600e+04,0.0,-127.00000,0.000,78.0,1.487510e+05,...,0.000,0.000000e+00,11267.00,0.820100,0.8201,34054.879,3.0,2022.0,1.0,Educational Services
3597,ZWS,2022q1,239600.0,137700.0,5.690000e+04,1100.0,1100.00000,4800.000,10000.0,1.118600e+06,...,5600.000,5.403000e+05,193700.00,35.281528,35.4000,125782.456,3.0,2022.0,1.0,Manufacturing
3598,ZY,2022q1,4791.0,12455.0,5.608200e+04,-130.0,-532.00000,7994.000,-26.0,6.181890e+05,...,57845.000,1.811680e+05,344064.00,2.890000,2.8900,103120.808,3.0,2022.0,1.0,"Professional, Scientific, and Technical Services"


In [8]:
# Get Market Value
df["Market Value"] = df["originalprice"] * df["sharesoutstanding"]
df[["Market Value"]]

,Market Value
0,3.971400e+07
1,1.669183e+07
2,1.215069e+05
3,1.184717e+07
4,6.378494e+04
...,...
3595,3.047556e+05
3596,2.792841e+04
3597,4.452699e+06
3598,2.980191e+05


In [9]:
# Get Book Value
df["Book Value"] = df["totalassets"] - df["totalliabilities"]
df[["Book Value"]]

,Book Value
0,5154000.00
1,6257000.00
2,218097.00
3,-8940000.00
4,126423.00
...,...
3595,93861.00
3596,11267.00
3597,193700.00
3598,344064.00


In [10]:
# Get Earning Before Interests and Taxes (Ebit)
df["Ebit"] = df["revenue"] - df["cogs"] - df["sgae"]
df[["Ebit"]]

,Ebit
0,376000.000
1,899000.000
2,3697.000
3,-1723000.000
4,3796.000
...,...
3595,-8654.000
3596,-7232.000
3597,45000.000
3598,-63746.000


In [11]:
# Get Operating Profit Margin
df["OPM"] = (df["Ebit"]/df["revenue"]).replace([np.inf, -np.inf], np.nan)
df[["OPM"]]

,OPM
0,0.224612
1,0.273003
2,0.436482
3,-0.193617
4,0.073554
...,...
3595,-0.227533
3596,-0.117340
3597,0.187813
3598,-13.305364


In [12]:
# Get Net Income
df["Net Income"] = df["Ebit"] - df["incometax"] - df["finexp"] + df["extraincome"]
df[["Net Income"]]

,Net Income
0,2.830000e+05
1,5.940000e+05
2,-2.701000e+03
3,-1.635000e+06
4,2.842000e+03
...,...
3595,-1.997000e+03
3596,-7.437000e+03
3597,3.130000e+04
3598,-7.224600e+04


In [13]:
# Get Prfit Margin
df["Profit Margin"] = ((df["Net Income"] / df["revenue"])*100).replace([np.inf, -np.inf], np.nan)
df[["Profit Margin"]]

,Profit Margin
0,16.905615
1,18.038263
2,-31.889020
3,-18.372851
4,5.506898
...,...
3595,-5.250565
3596,-12.066588
3597,13.063439
3598,-1507.952411


In [14]:
# Create earnings per share variable
EPS = df["Ebit"] / df["sharesoutstanding"]

In [15]:
# Create earnings per share deflated by price column
df["EPSP"] = EPS / df["originalprice"]
df[["EPSP"]]

,EPSP
0,0.009468
1,0.053859
2,0.030426
3,-0.145436
4,0.059512
...,...
3595,-0.028397
3596,-0.258948
3597,0.010106
3598,-0.213899


In [16]:
# Get sales annual growth
df["SAG"] = ((df["revenue"] / (df["revenue"].shift(-4)))-1).replace([np.inf, -np.inf], np.nan)
df[["SAG"]]

,SAG
0,31.436831
1,62.033575
2,-0.953658
3,NaN
4,-0.999469
...,...
3595,NaN
3596,NaN
3597,NaN
3598,NaN


In [17]:
# Get Operating Profit Growth
df["OPG"] = (df["Ebit"] / (df["Ebit"].shift(-4)))-1
df[["OPG"]]

,OPG
0,98.051633
1,-64.136456
2,-0.839317
3,NaN
4,-0.999873
...,...
3595,-0.028068
3596,NaN
3597,NaN
3598,NaN


In [18]:
# Get Book to Market Ratio 
df["BMR"] = df["Book Value"] / df["Market Value"]
df[["BMR"]]

,BMR
0,0.129778
1,0.374854
2,1.794935
3,-0.754610
4,1.982019
...,...
3595,0.307988
3596,0.403424
3597,0.043502
3598,1.154503


In [19]:
# Get short financial leverage 
df["SFL"] = df["shortdebt"] / df["totalassets"]
df[["SFL"]]

,SFL
0,0.000000
1,0.000063
2,NaN
3,0.035341
4,NaN
...,...
3595,0.005220
3596,0.000000
3597,0.005006
3598,0.093572


In [20]:
# Get long financial leverage
df["LFL"] = df["longdebt"] / df["totalassets"]
df[["LFL"]]

,LFL
0,0.264356
1,0.108019
2,0.178003
3,0.526120
4,0.089961
...,...
3595,0.004155
3596,0.000000
3597,0.483014
3598,0.293062


In [21]:
# Get dependant variable - stock returns
df["r"] = np.log(df["adjprice"]) - np.log(df["adjprice"].shift(4))
df[["r"]]

,r
0,NaN
1,NaN
2,NaN
3,NaN
4,-3.746430
...,...
3595,-3.966445
3596,-5.434321
3597,-0.079738
3598,-1.645459


In [22]:
df[["adjprice"]]

,adjprice
0,131.888962
1,89.694160
2,3.470000
3,18.250000
4,3.112825
...,...
3595,4.570000
3596,0.820100
3597,35.281528
3598,2.890000


In [23]:
df["F1r"] = df["r"].shift(-1)
df[["F1r"]]

,F1r
0,NaN
1,NaN
2,NaN
3,-3.746430
4,-3.201678
...,...
3595,-5.434321
3596,-0.079738
3597,-1.645459
3598,-0.801673


In [24]:
commercial = df.copy()

In [25]:
# Get data from commercial industry
commercial = commercial.loc[(commercial["Sector NAICS level 1"] == "Finance and Insurance") | (commercial["Sector NAICS level 1"] == "Real Estate and Rental and Leasing")]
commercial

,firm,q,revenue,cogs,sgae,otheropexp,extraincome,finexp,incometax,totalassets,...,Net Income,Profit Margin,EPSP,SAG,OPG,BMR,SFL,LFL,r,F1r
2,AAIC,2022q1,8470.0,4773.0,0.0,0.0,-4111.0,0.0,2287.0,920883.0,...,-2701.0,-31.889020,0.030426,-0.953658,-0.839317,1.794935,NaN,0.178003,NaN,NaN
4,AAME,2022q1,51608.0,0.0,47812.0,0.0,0.0,0.0,954.0,375031.0,...,2842.0,5.506898,0.059512,-0.999469,-0.999873,1.982019,NaN,0.089961,-3.746430,-3.201678
9,AAT,2022q1,101470.0,0.0,73128.0,0.0,-2998.0,14666.0,0.0,3011134.0,...,10678.0,10.523307,0.012359,-0.446650,-0.835740,0.525842,NaN,0.547644,2.330198,0.441623
13,ABCB,2022q1,183374.0,10830.0,0.0,0.0,-63140.0,0.0,27706.0,23560292.0,...,81698.0,44.552663,0.056434,-0.320460,1.654401,0.983560,NaN,0.023444,0.152854,-2.394947
20,ABR,2022q1,166698.0,82559.0,0.0,0.0,3978.0,0.0,8188.0,16308906.0,...,79929.0,47.948386,0.032423,6495.414653,-56.111025,1.068207,NaN,0.802576,-1.031596,-3.671046
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3520,WTM,2022q1,348900.0,0.0,361100.0,0.0,41800.0,0.0,-3800.0,7356700.0,...,33400.0,9.572944,-0.003578,44.932070,7.379121,1.076141,NaN,0.066443,3.503317,0.639115
3533,WY,2022q1,3112000.0,1647000.0,115000.0,6000.0,-291000.0,73000.0,209000.0,17116000.0,...,777000.0,24.967866,0.047727,NaN,-136.937972,0.361911,0.000000,0.295221,-1.193514,0.246446
3540,XELB,2022q1,8747.0,1680.0,10065.0,32.0,251.0,708.0,0.0,123709.0,...,-3455.0,-39.499257,-0.102809,-0.158457,-0.690257,2.442752,0.028656,0.258356,-3.227974,1.433444
3563,Y,2022q1,2718734.0,0.0,2504759.0,0.0,-43212.0,0.0,45082.0,31894427.0,...,125681.0,4.622777,0.018672,8.259110,5.427606,0.781984,NaN,0.088504,6.619483,0.424883


In [26]:
# Get size of each 
commercial["Size"] = pd.cut(commercial["Market Value"], bins=3, labels=["small", "medium", "big"])
commercial

,firm,q,revenue,cogs,sgae,otheropexp,extraincome,finexp,incometax,totalassets,...,Profit Margin,EPSP,SAG,OPG,BMR,SFL,LFL,r,F1r,Size
2,AAIC,2022q1,8470.0,4773.0,0.0,0.0,-4111.0,0.0,2287.0,920883.0,...,-31.889020,0.030426,-0.953658,-0.839317,1.794935,NaN,0.178003,NaN,NaN,small
4,AAME,2022q1,51608.0,0.0,47812.0,0.0,0.0,0.0,954.0,375031.0,...,5.506898,0.059512,-0.999469,-0.999873,1.982019,NaN,0.089961,-3.746430,-3.201678,small
9,AAT,2022q1,101470.0,0.0,73128.0,0.0,-2998.0,14666.0,0.0,3011134.0,...,10.523307,0.012359,-0.446650,-0.835740,0.525842,NaN,0.547644,2.330198,0.441623,small
13,ABCB,2022q1,183374.0,10830.0,0.0,0.0,-63140.0,0.0,27706.0,23560292.0,...,44.552663,0.056434,-0.320460,1.654401,0.983560,NaN,0.023444,0.152854,-2.394947,small
20,ABR,2022q1,166698.0,82559.0,0.0,0.0,3978.0,0.0,8188.0,16308906.0,...,47.948386,0.032423,6495.414653,-56.111025,1.068207,NaN,0.802576,-1.031596,-3.671046,small
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3520,WTM,2022q1,348900.0,0.0,361100.0,0.0,41800.0,0.0,-3800.0,7356700.0,...,9.572944,-0.003578,44.932070,7.379121,1.076141,NaN,0.066443,3.503317,0.639115,small
3533,WY,2022q1,3112000.0,1647000.0,115000.0,6000.0,-291000.0,73000.0,209000.0,17116000.0,...,24.967866,0.047727,NaN,-136.937972,0.361911,0.000000,0.295221,-1.193514,0.246446,small
3540,XELB,2022q1,8747.0,1680.0,10065.0,32.0,251.0,708.0,0.0,123709.0,...,-39.499257,-0.102809,-0.158457,-0.690257,2.442752,0.028656,0.258356,-3.227974,1.433444,small
3563,Y,2022q1,2718734.0,0.0,2504759.0,0.0,-43212.0,0.0,45082.0,31894427.0,...,4.622777,0.018672,8.259110,5.427606,0.781984,NaN,0.088504,6.619483,0.424883,small


In [27]:
dummies = pd.get_dummies(commercial["Size"], drop_first= -1)

In [28]:
commercial[["Medium"]] = dummies[["medium"]]
commercial[["Big"]] = dummies[["big"]]
commercial

,firm,q,revenue,cogs,sgae,otheropexp,extraincome,finexp,incometax,totalassets,...,SAG,OPG,BMR,SFL,LFL,r,F1r,Size,Medium,Big
2,AAIC,2022q1,8470.0,4773.0,0.0,0.0,-4111.0,0.0,2287.0,920883.0,...,-0.953658,-0.839317,1.794935,NaN,0.178003,NaN,NaN,small,0,0
4,AAME,2022q1,51608.0,0.0,47812.0,0.0,0.0,0.0,954.0,375031.0,...,-0.999469,-0.999873,1.982019,NaN,0.089961,-3.746430,-3.201678,small,0,0
9,AAT,2022q1,101470.0,0.0,73128.0,0.0,-2998.0,14666.0,0.0,3011134.0,...,-0.446650,-0.835740,0.525842,NaN,0.547644,2.330198,0.441623,small,0,0
13,ABCB,2022q1,183374.0,10830.0,0.0,0.0,-63140.0,0.0,27706.0,23560292.0,...,-0.320460,1.654401,0.983560,NaN,0.023444,0.152854,-2.394947,small,0,0
20,ABR,2022q1,166698.0,82559.0,0.0,0.0,3978.0,0.0,8188.0,16308906.0,...,6495.414653,-56.111025,1.068207,NaN,0.802576,-1.031596,-3.671046,small,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3520,WTM,2022q1,348900.0,0.0,361100.0,0.0,41800.0,0.0,-3800.0,7356700.0,...,44.932070,7.379121,1.076141,NaN,0.066443,3.503317,0.639115,small,0,0
3533,WY,2022q1,3112000.0,1647000.0,115000.0,6000.0,-291000.0,73000.0,209000.0,17116000.0,...,NaN,-136.937972,0.361911,0.000000,0.295221,-1.193514,0.246446,small,0,0
3540,XELB,2022q1,8747.0,1680.0,10065.0,32.0,251.0,708.0,0.0,123709.0,...,-0.158457,-0.690257,2.442752,0.028656,0.258356,-3.227974,1.433444,small,0,0
3563,Y,2022q1,2718734.0,0.0,2504759.0,0.0,-43212.0,0.0,45082.0,31894427.0,...,8.259110,5.427606,0.781984,NaN,0.088504,6.619483,0.424883,small,0,0


In [29]:
commercial.value_counts(["Big"])

Big
0      766
1        3
dtype: int64

In [30]:
commercial.value_counts(["Medium"])

Medium
0         767
1           2
dtype: int64

In [31]:
result = sm.ols(formula="F1r ~ EPSP + OPM + BMR + SFL + Medium + Big", data=commercial).fit()

In [32]:
print(result.summary())

                            OLS Regression Results                            
Dep. Variable:                    F1r   R-squared:                       0.041
Model:                            OLS   Adj. R-squared:                  0.007
Method:                 Least Squares   F-statistic:                     1.207
Date:                Sat, 26 Nov 2022   Prob (F-statistic):              0.305
Time:                        04:51:31   Log-Likelihood:                -382.43
No. Observations:                 177   AIC:                             778.9
Df Residuals:                     170   BIC:                             801.1
Df Model:                           6                                         
Covariance Type:            nonrobust                                         
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
Intercept     -0.1113      0.218     -0.510      0.6

### **1. Exploratory Analysis of Variables (Commercial Industry)**

#### ***1.1.1 Get Variance-Covariance Matrix of dependent and independent variables***

For this matter, we can say that the variance-covariance matrix is a square matrix that contains the variances and covariances associated with several variables, where one variable contains the variance of each variable in its diagonaland covariances between the variables in its non-diagonal terms. 

The formula of this matrix follows:

Where N is the number of data, 1 is a row of ONEs and X contains the variables, and ' indicates a transpose.


![Variance-Covariance Matrix formula](images/varcor.png)

In [33]:
df_f = commercial.copy()
df_f

,firm,q,revenue,cogs,sgae,otheropexp,extraincome,finexp,incometax,totalassets,...,SAG,OPG,BMR,SFL,LFL,r,F1r,Size,Medium,Big
2,AAIC,2022q1,8470.0,4773.0,0.0,0.0,-4111.0,0.0,2287.0,920883.0,...,-0.953658,-0.839317,1.794935,NaN,0.178003,NaN,NaN,small,0,0
4,AAME,2022q1,51608.0,0.0,47812.0,0.0,0.0,0.0,954.0,375031.0,...,-0.999469,-0.999873,1.982019,NaN,0.089961,-3.746430,-3.201678,small,0,0
9,AAT,2022q1,101470.0,0.0,73128.0,0.0,-2998.0,14666.0,0.0,3011134.0,...,-0.446650,-0.835740,0.525842,NaN,0.547644,2.330198,0.441623,small,0,0
13,ABCB,2022q1,183374.0,10830.0,0.0,0.0,-63140.0,0.0,27706.0,23560292.0,...,-0.320460,1.654401,0.983560,NaN,0.023444,0.152854,-2.394947,small,0,0
20,ABR,2022q1,166698.0,82559.0,0.0,0.0,3978.0,0.0,8188.0,16308906.0,...,6495.414653,-56.111025,1.068207,NaN,0.802576,-1.031596,-3.671046,small,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3520,WTM,2022q1,348900.0,0.0,361100.0,0.0,41800.0,0.0,-3800.0,7356700.0,...,44.932070,7.379121,1.076141,NaN,0.066443,3.503317,0.639115,small,0,0
3533,WY,2022q1,3112000.0,1647000.0,115000.0,6000.0,-291000.0,73000.0,209000.0,17116000.0,...,NaN,-136.937972,0.361911,0.000000,0.295221,-1.193514,0.246446,small,0,0
3540,XELB,2022q1,8747.0,1680.0,10065.0,32.0,251.0,708.0,0.0,123709.0,...,-0.158457,-0.690257,2.442752,0.028656,0.258356,-3.227974,1.433444,small,0,0
3563,Y,2022q1,2718734.0,0.0,2504759.0,0.0,-43212.0,0.0,45082.0,31894427.0,...,8.259110,5.427606,0.781984,NaN,0.088504,6.619483,0.424883,small,0,0


In [34]:
df_f = df_f.dropna()
df_f

,firm,q,revenue,cogs,sgae,otheropexp,extraincome,finexp,incometax,totalassets,...,SAG,OPG,BMR,SFL,LFL,r,F1r,Size,Medium,Big
25,AC,2022q1,2582.0000,0.00,5888.000,0.000,-2889.00000,14839.0,-4848.000,1.284192e+06,...,-0.972460,-0.910518,0.993543,0.000000,0.000000,1.600981,-0.719913,small,0,0
46,ACTG,2022q1,13507.0000,0.00,22015.000,0.000,-77035.00000,2601.0,-14878.000,6.614400e+05,...,-0.996831,-1.005385,1.648238,0.254991,0.003474,-1.208086,0.667291,small,0,0
68,AEI,2022q1,1952.2371,1114.55,2491.228,0.000,-4764.03080,-172.4,222.114,1.928306e+05,...,-0.998705,-1.017993,3.620261,0.001657,0.001792,-1.046556,0.809028,small,0,0
78,AFCG,2022q1,16935.7380,0.00,1144.444,5236.604,-374.28625,0.0,18.284,4.540676e+05,...,-0.996788,-0.987624,0.891454,0.000000,0.218182,1.635568,2.707001,small,0,0
83,AFRM,2022q1,354762.0000,15824.00,452342.000,113147.000,172139.00000,0.0,259.000,7.031726e+06,...,425.396635,0.121202,0.195679,0.370728,0.000000,-1.080566,4.584991,small,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3375,VIRT,2022q1,701262.0000,0.00,459551.000,0.000,-87668.00000,0.0,41786.000,1.251909e+07,...,426.338208,-45.164261,0.256526,0.011245,0.164879,1.657569,-1.687021,small,0,0
3414,VRTS,2022q1,252440.0000,0.00,186888.000,0.000,-22099.00000,-6341.0,16735.000,3.701866e+06,...,-0.883989,-0.803737,0.457551,0.000000,0.606282,2.674724,-0.472221,small,0,0
3457,WD,2022q1,319444.0000,0.00,223049.000,0.000,679.00000,6405.0,19460.000,4.339631e+06,...,8.600986,-8.135613,0.384601,0.000000,0.000000,3.698389,1.468105,small,0,0
3467,WETF,2022q1,78368.0000,0.00,60679.000,0.000,-41725.00000,2938.0,-16713.000,1.001037e+06,...,-0.932716,-0.870663,0.294466,0.000244,0.319127,-2.830512,0.625450,small,0,0


In [35]:
# Get size of dataset
size = df_f.shape

In [36]:
# Add ONES column
rows = size[:1]
one = np.ones([rows[0],1])

In [37]:
# Define variables
x_mat = df_f[["EPSP","OPM", "BMR", "SFL","Medium", "Big", "F1r"]]
x_mat

,EPSP,OPM,BMR,SFL,Medium,Big,F1r
25,-0.003577,-1.280403,0.993543,0.000000,0,0,-0.719913
46,-0.040591,-0.629896,1.648238,0.254991,0,0,0.667291
68,-0.031758,-0.846998,3.620261,0.001657,0,0,0.809028
78,0.041833,0.932424,0.891454,0.000000,0,0,2.707001
83,-0.008614,-0.319662,0.195679,0.370728,0,0,4.584991
...,...,...,...,...,...,...,...
3375,0.036501,0.344680,0.256526,0.011245,0,0,-1.687021
3414,0.036389,0.259674,0.457551,0.000000,0,0,-0.472221
3457,0.022645,0.301759,0.384601,0.000000,0,0,1.468105
3467,0.020558,0.225717,0.294466,0.000244,0,0,0.625450


In [38]:
# Transpose of variables
x_transp = x_mat.transpose()
x_transp

,25,46,68,78,83,116,134,141,174,177,...,3320,3340,3350,3365,3369,3375,3414,3457,3467,3540
EPSP,-0.003577,-0.040591,-0.031758,0.041833,-0.008614,0.016972,0.010917,0.127231,0.037847,0.022234,...,0.030568,0.004506,0.072096,-0.037921,0.019080,0.036501,0.036389,0.022645,0.020558,-0.102809
OPM,-1.280403,-0.629896,-0.846998,0.932424,-0.319662,0.255615,0.187373,0.773554,0.349415,0.245396,...,0.154279,0.280269,0.616249,-880.600000,0.975417,0.344680,0.259674,0.301759,0.225717,-0.342746
BMR,0.993543,1.648238,3.620261,0.891454,0.195679,0.246555,0.635080,0.139609,0.640866,0.632181,...,0.191884,0.124456,0.501638,1.573798,0.726966,0.256526,0.457551,0.384601,0.294466,2.442752
SFL,0.000000,0.254991,0.001657,0.000000,0.370728,0.009810,0.010191,0.000000,0.000000,0.000000,...,0.004954,0.009356,0.000000,0.000000,0.000000,0.011245,0.000000,0.000000,0.000244,0.028656
Medium,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
Big,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
F1r,-0.719913,0.667291,0.809028,2.707001,4.584991,-0.952788,-0.997084,-4.354876,0.778036,-1.872484,...,3.294068,-1.403295,-1.375812,-0.654176,0.879995,-1.687021,-0.472221,1.468105,0.625450,1.433444


In [39]:
# Multiply Transpose of varibles by ones column
transp_ones = x_transp.dot(one)
transp_ones

,0
EPSP,7.406486
OPM,-816.473245
BMR,105.675449
SFL,10.467622
Medium,1.000000
Big,1.000000
F1r,-12.734082


In [40]:
# Multiply transpose by ones by the transpose of the same variable
mult_transp = transp_ones.dot(transp_ones.transpose())
mult_transp

,EPSP,OPM,BMR,SFL,Medium,Big,F1r
EPSP,54.856036,-6047.197695,782.683742,77.528296,7.406486,7.406486,-94.314803
OPM,-6047.197695,666628.559492,-86281.177105,-8546.533286,-816.473245,-816.473245,10397.037512
BMR,782.683742,-86281.177105,11167.300615,1106.170658,105.675449,105.675449,-1345.679872
SFL,77.528296,-8546.533286,1106.170658,109.571110,10.467622,10.467622,-133.295560
Medium,7.406486,-816.473245,105.675449,10.467622,1.000000,1.000000,-12.734082
Big,7.406486,-816.473245,105.675449,10.467622,1.000000,1.000000,-12.734082
F1r,-94.314803,10397.037512,-1345.679872,-133.295560,-12.734082,-12.734082,162.156853


In [41]:
# Multiply Transpose of variables by varibales
mult_x = x_transp.dot(x_mat)
mult_x

,EPSP,OPM,BMR,SFL,Medium,Big,F1r
EPSP,4.311798,17.212828,12.663234,1.319344,0.011847,0.014484,-5.788116
OPM,17.212828,776335.660340,-1238.088067,2.764192,0.250595,0.086713,624.788058
BMR,12.663234,-1238.088067,190.802314,9.287409,0.105420,0.158656,-16.654793
SFL,1.319344,2.764192,9.287409,3.397014,0.000000,0.030271,2.768796
Medium,0.011847,0.250595,0.105420,0.000000,1.000000,0.000000,0.078950
Big,0.014484,0.086713,0.158656,0.030271,0.000000,1.000000,2.352086
F1r,-5.788116,624.788058,-16.654793,2.768796,0.078950,2.352086,722.469871


In [42]:
# Multiply fisrt multiplication by 1/size of data set
div = mult_transp*(1/rows[0])
div

,EPSP,OPM,BMR,SFL,Medium,Big,F1r
EPSP,0.349402,-38.517183,4.985247,0.493811,0.047175,0.047175,-0.600731
OPM,-38.517183,4246.041780,-549.561638,-54.436518,-5.200467,-5.200467,66.223169
BMR,4.985247,-549.561638,71.129303,7.045673,0.673092,0.673092,-8.571209
SFL,0.493811,-54.436518,7.045673,0.697905,0.066673,0.066673,-0.849016
Medium,0.047175,-5.200467,0.673092,0.066673,0.006369,0.006369,-0.081109
Big,0.047175,-5.200467,0.673092,0.066673,0.006369,0.006369,-0.081109
F1r,-0.600731,66.223169,-8.571209,-0.849016,-0.081109,-0.081109,1.032846


In [43]:
# Get Variance-Covariance Matrix
VarCov = (mult_x - div)*(1/(rows[0]-1))
VarCov

,EPSP,OPM,BMR,SFL,Medium,Big,F1r
EPSP,0.025400,0.357244,0.049218,0.005292,-0.000226,-0.000210,-0.033252
OPM,0.357244,4949.292427,-4.413631,0.366671,0.034943,0.033892,3.580544
BMR,0.049218,-4.413631,0.767135,0.014370,-0.003639,-0.003298,-0.051818
SFL,0.005292,0.366671,0.014370,0.017302,-0.000427,-0.000233,0.023191
Medium,-0.000226,0.034943,-0.003639,-0.000427,0.006369,-0.000041,0.001026
Big,-0.000210,0.033892,-0.003298,-0.000233,-0.000041,0.006369,0.015597
F1r,-0.033252,3.580544,-0.051818,0.023191,0.001026,0.015597,4.624596


In [44]:
# Check result with Python function
x_mat.cov()

,EPSP,OPM,BMR,SFL,Medium,Big,F1r
EPSP,0.025400,0.357244,0.049218,0.005292,-0.000226,-0.000210,-0.033252
OPM,0.357244,4949.292427,-4.413631,0.366671,0.034943,0.033892,3.580544
BMR,0.049218,-4.413631,0.767135,0.014370,-0.003639,-0.003298,-0.051818
SFL,0.005292,0.366671,0.014370,0.017302,-0.000427,-0.000233,0.023191
Medium,-0.000226,0.034943,-0.003639,-0.000427,0.006369,-0.000041,0.001026
Big,-0.000210,0.033892,-0.003298,-0.000233,-0.000041,0.006369,0.015597
F1r,-0.033252,3.580544,-0.051818,0.023191,0.001026,0.015597,4.624596


WITH THIS WE CAN SAY THAT THE CALCULATIONS MADE WITH MATRICIAL ALGEBRA GAVE THE SAME RESULT AS THE ONES THAT PYTHON SHOWED, WICH MEANS THE CALCULATIONS WERE CORRECT. 

#### ***1.1.2 Get Correlation matrix of dependent and independent variables***

The correlation matrix is a table which dispays the correlation coefficients for different variables.

The formula follows:

Where σ is the standard deviation and Cov indicates the diagonal of the Variance-Covariance matrix.

![Variance-Covariance Matrix formula](images/corr.png)

In [45]:
# Get diagonal of Variance-Covariance matrix
variance = np.diag(VarCov).reshape(1,7)

In [46]:
# Get the transpose of diagonal
var_transp = variance.transpose()
# Get sqrt of diagonal
var_sqrt = np.sqrt(variance)

In [47]:
# Get the standard deviation
std_deviation = var_sqrt * (np.sqrt(var_transp))

In [48]:
# Get correlation matrix
c_matrix = VarCov / std_deviation
c_matrix

,EPSP,OPM,BMR,SFL,Medium,Big,F1r
EPSP,1.000000,0.031862,0.352590,0.252433,-0.017804,-0.016475,-0.097022
OPM,0.031862,1.000000,-0.071629,0.039624,0.006224,0.006036,0.023667
BMR,0.352590,-0.071629,1.000000,0.124731,-0.052058,-0.047176,-0.027511
SFL,0.252433,0.039624,0.124731,1.000000,-0.040712,-0.022228,0.081985
Medium,-0.017804,0.006224,-0.052058,-0.040712,1.000000,-0.006410,0.005978
Big,-0.016475,0.006036,-0.047176,-0.022228,-0.006410,1.000000,0.090879
F1r,-0.097022,0.023667,-0.027511,0.081985,0.005978,0.090879,1.000000


In [49]:
# Check with Python function
x_mat.corr()

,EPSP,OPM,BMR,SFL,Medium,Big,F1r
EPSP,1.000000,0.031862,0.352590,0.252433,-0.017804,-0.016475,-0.097022
OPM,0.031862,1.000000,-0.071629,0.039624,0.006224,0.006036,0.023667
BMR,0.352590,-0.071629,1.000000,0.124731,-0.052058,-0.047176,-0.027511
SFL,0.252433,0.039624,0.124731,1.000000,-0.040712,-0.022228,0.081985
Medium,-0.017804,0.006224,-0.052058,-0.040712,1.000000,-0.006410,0.005978
Big,-0.016475,0.006036,-0.047176,-0.022228,-0.006410,1.000000,0.090879
F1r,-0.097022,0.023667,-0.027511,0.081985,0.005978,0.090879,1.000000


SOME OF THE BIGGEST AND HIGHEST CORRELATIONS ARE THE BOOK TO MARKET RATIO AGAINST EARNINGS PER SHARE, THIS VALUE IS POSITIVE AND SIGNIFICANT, EVEN IF ITS NOT HIGH IT MAKES SENSE THAT THESE TWO ARE CORRELATED ALSO MEANING THAT THEY BOTH DEPEND ON THE OTHER. ONE OF THE OTHER VARIABLES WITH HIGH CORRELATION ARE EARNINGS PER SHARE AGAINS SHORT LEVERAGE, THIS VALUE IS POSITVE AND SLIGHTLY SIGNIFICANT, MEANING THAT THE VALUE OF ONE DEPENDS ON THE OTHER. 

##### **1.1.3 Explain what is Variance, Covariance and Correlation**

- **VARIANCE:** VARIANCE IS A VALUE OR A MEASURE OF DISPERSION THAT INDICATES HOW MUCH A VARIABLE CHANGES, OR THE DIFFERENCE BETWEEN THEM. IN SIMPLE TERMS, IS THE AVERAGE SQUARED DEVIATIONS OF THE MEAN.

- **COVARIANCE:** AS THE NAME SAYS, IT INDICATES HOW MUCH AND X VALUE CO-MOVES. IT DEFINES A CHANGE BETWEEN TWO VARIABLES, AND IT MEASURES THE RELATIONSHIP BETWEEN THEM.

- **CORRELATION:** THIS EXPRESESSES HOW MUCH CERTAIN VARIABLES ARE RELATED TO ONE ANOTHER, HOWEVER IT DOESNT MEASSURE SENSIBILITY.

#### ***1.2.1 Leverage Points Detection (Unusual X Values)***

An outlier is a data point whose response does not follow the general trend of the rest of the data.

A data point has high leverage if it has "extreme" predictor values.

Considering the matrix formulation of regression model formulas as well as the predicted responses representations in a mtrix anotation, the estimated coefficients, then we can see the predicted responses as the following:


![Predicted responses formula](images/pr.png)

Where H matrix is defined as:

![H matrix formula](images/hmat.png)

Leaving:

![Hat matrix formula](images/hatmat.png)

this last one also known as "Hat-Matrix"

In [50]:
# Get shape of Dataset
sx = df_f.shape
rowsx = sx[:1]
ones = np.ones([rowsx[0],1])

In [51]:
# add ONEs column
df_f.insert(loc=1,column="ones", value=ones)
df_f

,firm,ones,q,revenue,cogs,sgae,otheropexp,extraincome,finexp,incometax,...,SAG,OPG,BMR,SFL,LFL,r,F1r,Size,Medium,Big
25,AC,1.0,2022q1,2582.0000,0.00,5888.000,0.000,-2889.00000,14839.0,-4848.000,...,-0.972460,-0.910518,0.993543,0.000000,0.000000,1.600981,-0.719913,small,0,0
46,ACTG,1.0,2022q1,13507.0000,0.00,22015.000,0.000,-77035.00000,2601.0,-14878.000,...,-0.996831,-1.005385,1.648238,0.254991,0.003474,-1.208086,0.667291,small,0,0
68,AEI,1.0,2022q1,1952.2371,1114.55,2491.228,0.000,-4764.03080,-172.4,222.114,...,-0.998705,-1.017993,3.620261,0.001657,0.001792,-1.046556,0.809028,small,0,0
78,AFCG,1.0,2022q1,16935.7380,0.00,1144.444,5236.604,-374.28625,0.0,18.284,...,-0.996788,-0.987624,0.891454,0.000000,0.218182,1.635568,2.707001,small,0,0
83,AFRM,1.0,2022q1,354762.0000,15824.00,452342.000,113147.000,172139.00000,0.0,259.000,...,425.396635,0.121202,0.195679,0.370728,0.000000,-1.080566,4.584991,small,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3375,VIRT,1.0,2022q1,701262.0000,0.00,459551.000,0.000,-87668.00000,0.0,41786.000,...,426.338208,-45.164261,0.256526,0.011245,0.164879,1.657569,-1.687021,small,0,0
3414,VRTS,1.0,2022q1,252440.0000,0.00,186888.000,0.000,-22099.00000,-6341.0,16735.000,...,-0.883989,-0.803737,0.457551,0.000000,0.606282,2.674724,-0.472221,small,0,0
3457,WD,1.0,2022q1,319444.0000,0.00,223049.000,0.000,679.00000,6405.0,19460.000,...,8.600986,-8.135613,0.384601,0.000000,0.000000,3.698389,1.468105,small,0,0
3467,WETF,1.0,2022q1,78368.0000,0.00,60679.000,0.000,-41725.00000,2938.0,-16713.000,...,-0.932716,-0.870663,0.294466,0.000244,0.319127,-2.830512,0.625450,small,0,0


In [52]:
# Define X variables
x1 = df_f[[ "ones", "EPSP","OPM", "BMR", "SFL","Medium", "Big"]]
x1

,ones,EPSP,OPM,BMR,SFL,Medium,Big
25,1.0,-0.003577,-1.280403,0.993543,0.000000,0,0
46,1.0,-0.040591,-0.629896,1.648238,0.254991,0,0
68,1.0,-0.031758,-0.846998,3.620261,0.001657,0,0
78,1.0,0.041833,0.932424,0.891454,0.000000,0,0
83,1.0,-0.008614,-0.319662,0.195679,0.370728,0,0
...,...,...,...,...,...,...,...
3375,1.0,0.036501,0.344680,0.256526,0.011245,0,0
3414,1.0,0.036389,0.259674,0.457551,0.000000,0,0
3457,1.0,0.022645,0.301759,0.384601,0.000000,0,0
3467,1.0,0.020558,0.225717,0.294466,0.000244,0,0


In [53]:
# Get transpose of X variables
x_transpl = x1.transpose()
x_transpl

,25,46,68,78,83,116,134,141,174,177,...,3320,3340,3350,3365,3369,3375,3414,3457,3467,3540
ones,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,...,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000
EPSP,-0.003577,-0.040591,-0.031758,0.041833,-0.008614,0.016972,0.010917,0.127231,0.037847,0.022234,...,0.030568,0.004506,0.072096,-0.037921,0.019080,0.036501,0.036389,0.022645,0.020558,-0.102809
OPM,-1.280403,-0.629896,-0.846998,0.932424,-0.319662,0.255615,0.187373,0.773554,0.349415,0.245396,...,0.154279,0.280269,0.616249,-880.600000,0.975417,0.344680,0.259674,0.301759,0.225717,-0.342746
BMR,0.993543,1.648238,3.620261,0.891454,0.195679,0.246555,0.635080,0.139609,0.640866,0.632181,...,0.191884,0.124456,0.501638,1.573798,0.726966,0.256526,0.457551,0.384601,0.294466,2.442752
SFL,0.000000,0.254991,0.001657,0.000000,0.370728,0.009810,0.010191,0.000000,0.000000,0.000000,...,0.004954,0.009356,0.000000,0.000000,0.000000,0.011245,0.000000,0.000000,0.000244,0.028656
Medium,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
Big,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000


In [54]:
# Multiply transpose of variables by the X variables
hm = np.dot(x_transpl,x1)

In [55]:
# Get inverse of matrix
inv = np.linalg.inv(hm)

In [56]:
# Multiply X variables by the inverse 
hm_m = x1.dot(inv)

In [57]:
# Get Hat Matrix
hat_mat = np.dot(hm_m,x_transpl)
hat_mat

array([[ 0.01023417,  0.0075369 ,  0.02162806, ...,  0.00728388,
         0.00693465,  0.01723016],
       [ 0.0075369 ,  0.03676809,  0.03708597, ..., -0.00042593,
        -0.00123614,  0.03099354],
       [ 0.02162806,  0.03708597,  0.10273644, ...,  0.00144378,
        -0.00112706,  0.07175511],
       ...,
       [ 0.00728388, -0.00042593,  0.00144378, ...,  0.00869586,
         0.00887989,  0.00315679],
       [ 0.00693465, -0.00123614, -0.00112706, ...,  0.00887989,
         0.00913732,  0.00151116],
       [ 0.01723016,  0.03099354,  0.07175511, ...,  0.00315679,
         0.00151116,  0.05339758]])

In [58]:
# Since the leverages are the diagonal of the Hat Matrix
# Get leverages of Hat Matrix
leverages = np.diagonal(hat_mat)
leverages

array([0.01023417, 0.03676809, 0.10273644, 0.00885251, 0.04856357,
       0.00900812, 0.00784499, 0.01520459, 0.00822934, 0.00821043,
       0.00988798, 0.00997179, 0.00998887, 0.00897437, 0.00922899,
       0.00883909, 0.00876329, 0.00833411, 0.01867511, 0.00895673,
       0.0087331 , 0.00780316, 0.01720741, 0.00948069, 0.01948   ,
       0.009108  , 0.01011699, 0.01381524, 0.00915449, 0.00826165,
       0.00999646, 0.00756631, 0.00756675, 0.00816307, 0.00894677,
       0.00923851, 0.0109688 , 0.42077484, 0.00997866, 0.00837904,
       0.08120996, 0.24311697, 0.01691968, 0.01082851, 0.01119357,
       0.0092744 , 0.07317311, 0.00871139, 0.00847756, 0.12772708,
       0.01102752, 0.03106104, 0.00793733, 0.03246807, 0.00824457,
       0.00699735, 0.0095222 , 0.0107053 , 0.01037121, 0.00855551,
       0.11151226, 0.01167855, 0.00831221, 0.0090786 , 0.0067127 ,
       0.01147604, 0.01539736, 0.56649455, 0.03174699, 0.01070339,
       0.00863058, 0.00944449, 0.0935789 , 0.01044247, 0.00921

In [59]:
# Sum leverages to prove result
lev_sum = leverages.sum()
lev_sum

6.999999999999998

In [60]:
# Get mean of Leverages
lev_mean = lev_sum/leverages.shape[0]
lev_mean

0.04458598726114649

In [61]:
# Get extreme values of X
extreme_x = leverages > (3 * lev_mean)
extreme_x = leverages[extreme_x]
extreme_x

array([0.42077484, 0.24311697, 0.56649455, 0.14229796, 1.        ,
       0.14526818, 1.        , 0.99927985])

TO IDENTIFY THE DATA POINTS WHOSE VALUES ARE EXTREME, WE FOLLOW THE RULE WHERE WE FLAG ANY OBSERVATION WHOSE LEVERAGE VALUE, IS MORE THAN 3 TIMES LARGER THAN THE MEAN LEVERAGE VALUE. 

#### ***1.2.3 Outliers Detection (Unusual Y Values)***

There are two measures that we can use to identify outliers, they are: residuals and standarized resiuduals.

- Residuals: Defined for each observation as the difference between the observed and predicted responses.

- Standarized residuals: Defined for each observation as an ordinary residual divided by an estimate of its standard deviation. The formula follows:

    ![standarized formula](images/stdr.png)

In [62]:
out = df_f.copy()
out = df_f.rename(columns={"F1r" : "y"})

In [63]:
# Get Y column
out = out[["y"]]
out

,y
25,-0.719913
46,0.667291
68,0.809028
78,2.707001
83,4.584991
...,...
3375,-1.687021
3414,-0.472221
3457,1.468105
3467,0.625450


In [64]:
# Get predicted responses
out["FITS1"] = hat_mat.dot(out["y"])
out

,y,FITS1
25,-0.719913,-0.124755
46,0.667291,0.434092
68,0.809028,-0.007984
78,2.707001,-0.204459
83,4.584991,0.560196
...,...,...
3375,-1.687021,-0.190126
3414,-0.472221,-0.206145
3457,1.468105,-0.184062
3467,0.625450,-0.182250


In [65]:
# Get ordinary residual
out["RESI1"] = out["y"] - out["FITS1"]
out

,y,FITS1,RESI1
25,-0.719913,-0.124755,-0.595158
46,0.667291,0.434092,0.233199
68,0.809028,-0.007984,0.817012
78,2.707001,-0.204459,2.911460
83,4.584991,0.560196,4.024795
...,...,...,...
3375,-1.687021,-0.190126,-1.496895
3414,-0.472221,-0.206145,-0.266076
3457,1.468105,-0.184062,1.652166
3467,0.625450,-0.182250,0.807700


In [66]:
# Get leverages hii
out["HI1"] = leverages
out

,y,FITS1,RESI1,HI1
25,-0.719913,-0.124755,-0.595158,0.010234
46,0.667291,0.434092,0.233199,0.036768
68,0.809028,-0.007984,0.817012,0.102736
78,2.707001,-0.204459,2.911460,0.008853
83,4.584991,0.560196,4.024795,0.048564
...,...,...,...,...
3375,-1.687021,-0.190126,-1.496895,0.009103
3414,-0.472221,-0.206145,-0.266076,0.008528
3457,1.468105,-0.184062,1.652166,0.008696
3467,0.625450,-0.182250,0.807700,0.009137


In [84]:
# Standarized Residual calculations
res = (1 - out["HI1"])
MSE = (out["y"] - out["FITS1"])
MSE = np.square(MSE)
MSE = MSE.mean()
mult = MSE*res

In [85]:
# Get Standarized Residuals
out["SRES1"] = (out["RESI1"])/(np.sqrt(mult))
out

,y,FITS1,RESI1,HI1,SRES1,SRES1_abs
25,-0.719913,-0.124755,-0.595158,0.010234,-0.283434,0.283434
46,0.667291,0.434092,0.233199,0.036768,0.112576,0.112576
68,0.809028,-0.007984,0.817012,0.102736,0.408653,0.408653
78,2.707001,-0.204459,2.911460,0.008853,1.385568,1.385568
83,4.584991,0.560196,4.024795,0.048564,1.954970,1.954970
...,...,...,...,...,...,...
3375,-1.687021,-0.190126,-1.496895,0.009103,-0.712465,0.712465
3414,-0.472221,-0.206145,-0.266076,0.008528,-0.126605,0.126605
3457,1.468105,-0.184062,1.652166,0.008696,0.786206,0.786206
3467,0.625450,-0.182250,0.807700,0.009137,0.384441,0.384441


In [86]:
# if Standarized residual is larger than 3 (in absolute value) then = outlier
out["SRES1_abs"] = out["SRES1"].abs()
outlier = out[out["SRES1_abs"] > 3]
outlier

,y,FITS1,RESI1,HI1,SRES1,SRES1_abs


The column named "FITS1" contains the predicted responses, the column labeled "RESI1" contains the ordinary residuals, the column labeled "HI1" contains the leverages hii, and the column labeled "SRES1" contains the standardized residuals.

In [70]:
# Visualize outliers
import plotly.express as px
fig = px.box(out, y='y')
fig = fig.show()
fig

#### ***2. Multicollienarity Analysis***

Multicollienarity occurs when several independent variables are correlated. If we have this problem then the estimations of the stadndard errors of the beta coefficient will be unreliable because it becomes inflated. 

To check for multicollienarity issues, we follow the next steps:

1. Calculate the VIF (Variance Inflation Factor) for each variable X.
2. If VIF > 10 then we have a multicollienarity problem.
3. Drop the variable with highest VIF and repeat the process. 

In [71]:
# Create dataframe with Variables
data = {'variable':["EPSP","OPM", "BMR", "SFL","Medium", "Big", "ones"]}
VIF = pd.DataFrame(data)
VIF

,variable
0,EPSP
1,OPM
2,BMR
3,SFL
4,Medium
5,Big
6,ones


In [72]:
# Calculating VIF for each variable
VIF["VIF"] = [variance_inflation_factor(x1.values, i) for i in range(len(x1.columns))]
VIF

,variable,VIF
0,EPSP,1.807035
1,OPM,1.205895
2,BMR,1.010294
3,SFL,1.157828
4,Medium,1.072828
5,Big,1.004094
6,ones,1.002620


##### **Interpretation:**

CONSIDERING THE THINGS MENTIONED ABOVE WE CAN SAY THAT FOR THIS CASE WE DONT HAVE A MULTICOLLIENARITY PROBLEM BECAUSE ALL THE VARIABLES HAVE A VIF LOWER THAN 10, SO WE DONT HACE STRONG COLLIENARITY BETWEEN VARIABLES.

#### ***3. Solution to problems above***

To solve the problems mentioned above firstly we need to identify the outliers and leverage points and where this points intersect to drop them. Then there is a recalculation of this values and the process repits until there are no matches left.

In [73]:
# Get index of outliers and leverage points
out_idx = outlier.index
lev_idx = np.nonzero(leverages > (3 * lev_mean))

In [74]:
arr = np.intersect1d(out_idx, lev_idx)
arr

array([], dtype=int64)

In [75]:
# Drop intesections of leverage and outliers
df_f = df_f.drop(arr, axis=0)
df_f

,firm,ones,q,revenue,cogs,sgae,otheropexp,extraincome,finexp,incometax,...,SAG,OPG,BMR,SFL,LFL,r,F1r,Size,Medium,Big
25,AC,1.0,2022q1,2582.0000,0.00,5888.000,0.000,-2889.00000,14839.0,-4848.000,...,-0.972460,-0.910518,0.993543,0.000000,0.000000,1.600981,-0.719913,small,0,0
46,ACTG,1.0,2022q1,13507.0000,0.00,22015.000,0.000,-77035.00000,2601.0,-14878.000,...,-0.996831,-1.005385,1.648238,0.254991,0.003474,-1.208086,0.667291,small,0,0
68,AEI,1.0,2022q1,1952.2371,1114.55,2491.228,0.000,-4764.03080,-172.4,222.114,...,-0.998705,-1.017993,3.620261,0.001657,0.001792,-1.046556,0.809028,small,0,0
78,AFCG,1.0,2022q1,16935.7380,0.00,1144.444,5236.604,-374.28625,0.0,18.284,...,-0.996788,-0.987624,0.891454,0.000000,0.218182,1.635568,2.707001,small,0,0
83,AFRM,1.0,2022q1,354762.0000,15824.00,452342.000,113147.000,172139.00000,0.0,259.000,...,425.396635,0.121202,0.195679,0.370728,0.000000,-1.080566,4.584991,small,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3375,VIRT,1.0,2022q1,701262.0000,0.00,459551.000,0.000,-87668.00000,0.0,41786.000,...,426.338208,-45.164261,0.256526,0.011245,0.164879,1.657569,-1.687021,small,0,0
3414,VRTS,1.0,2022q1,252440.0000,0.00,186888.000,0.000,-22099.00000,-6341.0,16735.000,...,-0.883989,-0.803737,0.457551,0.000000,0.606282,2.674724,-0.472221,small,0,0
3457,WD,1.0,2022q1,319444.0000,0.00,223049.000,0.000,679.00000,6405.0,19460.000,...,8.600986,-8.135613,0.384601,0.000000,0.000000,3.698389,1.468105,small,0,0
3467,WETF,1.0,2022q1,78368.0000,0.00,60679.000,0.000,-41725.00000,2938.0,-16713.000,...,-0.932716,-0.870663,0.294466,0.000244,0.319127,-2.830512,0.625450,small,0,0


In [76]:
# Recalculation of Hat Matrix
x1 = df_f[["EPSP","OPM", "BMR", "SFL","Medium", "Big", "ones"]]
x_transpl = x1.transpose()
hm = np.dot(x_transpl,x1)
inv = np.linalg.inv(hm)
hm_m = x1.dot(inv)
hat_mat = np.dot(hm_m,x_transpl)
hat_mat


array([[ 0.01023417,  0.0075369 ,  0.02162806, ...,  0.00728388,
         0.00693465,  0.01723016],
       [ 0.0075369 ,  0.03676809,  0.03708597, ..., -0.00042593,
        -0.00123614,  0.03099354],
       [ 0.02162806,  0.03708597,  0.10273644, ...,  0.00144378,
        -0.00112706,  0.07175511],
       ...,
       [ 0.00728388, -0.00042593,  0.00144378, ...,  0.00869586,
         0.00887989,  0.00315679],
       [ 0.00693465, -0.00123614, -0.00112706, ...,  0.00887989,
         0.00913732,  0.00151116],
       [ 0.01723016,  0.03099354,  0.07175511, ...,  0.00315679,
         0.00151116,  0.05339758]])

In [77]:
# Get leverages and mean
leverages = np.diagonal(hat_mat)
lev_mean = lev_sum/leverages.shape[0]

In [78]:
# Recalculate outliers
out = df_f.copy()
out = df_f.rename(columns={"F1r" : "y"})
out = out[["y"]]
out["FITS1"] = hat_mat.dot(out["y"])
out["RESI1"] = out["y"] - out["FITS1"]
out["HI1"] = leverages
res = (1 - out["HI1"])
MSE = (np.square(out["y"] - out["FITS1"])).mean()
mult = MSE*res
out["SRES1"] = (out["RESI1"])/(np.sqrt(mult))
out["SRES1_abs"] = out["SRES1"].abs()
outlier = out[out["SRES1_abs"] > 3]

In [79]:
# Get index of outliers and leverage points
out_idx = outlier.index
lev_idx = np.nonzero(leverages > (3 * lev_mean))

In [80]:
arr = np.intersect1d(out_idx, lev_idx)
arr

array([], dtype=int64)

SINCE THERE ARE NO MORE INTERSECTIONS, THERE IS NO NEED TO REPEAT THE PROCESS. 

In [81]:
import statsmodels.formula.api as sm
result = sm.ols(formula="F1r ~ EPSP + OPM + BMR + SFL + Medium + Big", data=df_f).fit()

In [82]:
print(result.summary())

                            OLS Regression Results                            
Dep. Variable:                    F1r   R-squared:                       0.031
Model:                            OLS   Adj. R-squared:                 -0.008
Method:                 Least Squares   F-statistic:                    0.7876
Date:                Sat, 26 Nov 2022   Prob (F-statistic):              0.581
Time:                        04:51:40   Log-Likelihood:                -340.05
No. Observations:                 157   AIC:                             694.1
Df Residuals:                     150   BIC:                             715.5
Df Model:                           6                                         
Covariance Type:            nonrobust                                         
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
Intercept     -0.1544      0.232     -0.667      0.5

##### **Model Interpretation:**

THIS VALUE REPRENTS HOW MUCH VARIANCE IS EXPLAINED BY THE MODEL, OR THE PROPORTION OF VARIANCE THAT THE DEPENDANT VARIABLES CAN BE EXPLAINED BY THE INDEPENDENT ONES. CONSIDERING THE CHANGES MADE, THE RESULTS OF THE MODEL SHOW A R SQUARED VALUE OF 0.031, INDICATING THAT 3.1% PROPORTION OF VARIANCE THAT THE ANNUAL STOCK RETURN WILL BE EXPLAINED BY VARIABLES LIKE EARNINGS PER SHARE, OPERATING PROFIT MARGIN, BOOK MARKET RATIO, AND THE SIZED OF THE COMPANIES. THIS IS A VALID VALUE DUE TO THE FACT THAT THE MARKET IS CONSTANTLY CHANGING. 

WE CAN ALSO SEE OTHER VALUES, THAT CAN EXPLAIN THE RESULTS OF THE MODEL. FOR EXAMPLE, AS FOR THE T VALUE REPRESENTS THE DIFFERENCE RELATIVE TO THE VATIATION ON THE DATA, IT ALSO QUANTIFIES THE RELATIONSHIP BETWEEN THE INDEPENDENT AND DEPENDENT VARIABLE. THIS VALUE HAS A HIGH EXPLACATIVE POWER, IN INDICATES HOW MANY STANDARD DEVIATIONS THE VARIABLE IS AWAY FROM THE NULL HYPOTHESIS. WE ARE LOOKING FOT THIS VALUE TO BE HIGHER OR THE SAME AS 2 FOR IT TO REPRESENT A 95% OF PROBABILITY UNDER DE CURVE FOR THE HYPOTHESIS TO BE TRUE. IN THIS CASE ONLY TWO VARIABLES HAVE THIS CONDITION, THIS BEING THE EARNINGS PER SHARE AND THE OPERATING PROFIT MARGIN. 

AS FOR THE COEFFICIENTS, 2 ARE NEGATIVE, THE EARNINGS PER SHARE, INTERCEPT, THIS MEANS THIS VARIABLES HAVE A POSSITIVE IMPACT ON THE PREDICTION, AND THE OPPOSITTE FOR THE REST OF THE VARIABLES, SINCE THEY ARE NEGATIVE. 